In [86]:
import dask
import dask.dataframe as dd
from dask.delayed import delayed
import pandas as pd

parts = dask.delayed(pd.read_excel)("data.xlsx", sheet_name="2019", usecols = [0, 20, 55, 56, 57, 58])
df2019 = dd.from_delayed(parts)

parts = dask.delayed(pd.read_excel)("data.xlsx", sheet_name="2020", usecols = [0, 20, 55, 56, 57, 58])
df2020 = dd.from_delayed(parts)

parts = dask.delayed(pd.read_excel)("data.xlsx", sheet_name="2021", usecols = [0, 20, 56, 57, 58, 59])
df2021 = dd.from_delayed(parts)

parts = dask.delayed(pd.read_excel)("data.xlsx", sheet_name="2022", usecols = [0, 20, 55, 56, 57, 58])
df2022 = dd.from_delayed(parts)



In [87]:
def do_stuff(df, n):
  new_columns = ['Date', 'turbidity', 'chlorine', 'alum', 'lime_pre', 'lime_post']
  df = df.rename(columns=dict(zip(df.columns, new_columns)))
  df = df.head(n)
  df = df.drop(0).drop(1).reset_index(drop=True)
  df['Date'] = dd.to_datetime(df['Date'])
  df['Date'] = df['Date'].dt.floor('D')
  return df

df2019 = do_stuff(df2019, 367)
df2020 = do_stuff(df2020, 368)
df2021 = do_stuff(df2021, 367)
df2022 = do_stuff(df2022, 367)

print(df2019.head())
print(df2020.head())
print(df2021.head())
print(df2022.head())

        Date turbidity  chlorine   alum lime_pre lime_post
0 2019-01-01         0  2.991784      0        5         9
1 2019-01-02     0.401  2.876455  28.49        5         9
2 2019-01-03     0.374  4.180292  30.32        5         9
3 2019-01-04     0.361  3.144026  27.95        5         9
4 2019-01-05     0.351  2.930639  32.14      5.2       9.4
        Date turbidity  chlorine   alum lime_pre lime_post
0 2020-01-01     0.281  2.601027  28.52      5.6       9.9
1 2020-01-02     0.283  2.932847  27.81      5.6      10.1
2 2020-01-03     0.427  2.894292  31.45      5.6      10.3
3 2020-01-04     0.419       NaN  30.98      5.6      10.3
4 2020-01-05         0       NaN      0        0         0
        Date turbidity chlorine   alum lime_pre lime_post
0 2021-01-01     0.471   2.8888  30.56      4.8       9.3
1 2021-01-02     0.418  3.01435  30.38      4.8       9.3
2 2021-01-03       NaN      NaN    NaN      NaN       NaN
3 2021-01-04     0.534  2.70786  31.92      4.8       9.3
4 

In [96]:
data = dd.multi.concat([df2019, df2020, df2021, df2022])
data = data.reset_index(drop=True)
data["turbidity"] = data["turbidity"].apply(str).apply(lambda x: x.replace(',','0.'))
data['turbidity']=data['turbidity'].astype(float)
data['chlorine']=data['chlorine'].astype(float)
data['alum']=data['alum'].astype(float)
data['lime_pre']=data['lime_pre'].astype(float)
data['lime_post']=data['lime_post'].astype(float)

data.to_csv("data.csv", single_file = True)
data.head()

/usr/local/lib/python3.9/dist-packages/dask/dataframe/core.py:4134: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  Before: .apply(func)
  After:  .apply(func, meta=('turbidity', 'object'))

  warnings.warn(meta_warning(meta))


,Date,turbidity,chlorine,alum,lime_pre,lime_post
0,2019-01-01,0.000,2.991784,0.00,5.0,9.0
1,2019-01-02,0.401,2.876455,28.49,5.0,9.0
2,2019-01-03,0.374,4.180292,30.32,5.0,9.0
3,2019-01-04,0.361,3.144026,27.95,5.0,9.0
4,2019-01-05,0.351,2.930639,32.14,5.2,9.4


In [97]:
data.dtypes

Date         datetime64[ns]
turbidity           float64
chlorine            float64
alum                float64
lime_pre            float64
lime_post           float64
dtype: object